In [24]:
import os
from dotenv import load_dotenv
import json
import pandas as pd
import traceback

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [25]:
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [27]:
llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GEMINI_API_KEY)

In [28]:
from langchain_core.prompts import PromptTemplate

In [30]:
TEMPLATE = """
Text : {text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs

###RESPONSE JSON
{response_j son}"

"""

In [31]:
RESPONSE_JSON = {
    "1":  {
        "MCQ": "Multiple Choice Question",
        "Options": {"a": "Choice here", "b": "Choice here", "c": "Choice here", "d": "Choice here"},
        "Correct": "Correct Answer"
    },
    "2":  {
        "MCQ": "Multiple Choice Question",
        "Options": {"a": "Choice here", "b": "Choice here", "c": "Choice here", "d": "Choice here"},
        "Correct": "Correct Answer"
    },
    "3":  {
        "MCQ": "Multiple Choice Question",
        "Options": {"a": "Choice here", "b": "Choice here", "c": "Choice here", "d": "Choice here"},
        "Correct": "Correct Answer"
    }
}

In [32]:
quiz_prompt =  PromptTemplate(template=TEMPLATE, input_variables=["text", "number", "subject", "tone", "response_json"])